In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
from pgpq import ArrowToPostgresBinaryEncoder

In [2]:
arrow_table = pq.read_table("tests/testdata/yellow_tripdata_2022-01.parquet")

# use just the first row until https://github.com/apache/arrow-rs/issues/3646 gets resolved
batch = pa.RecordBatch.from_pylist(next(iter(arrow_table.slice(0, 1).to_batches())).to_pylist())
small_batches = [pa.RecordBatch.from_pylist(batch.to_pylist() * 100)]
large_batches = [pa.RecordBatch.from_pylist(batch.to_pylist() * 10_000) for _ in range(10_000)]  # 10k batches of 10k rows = 100m

In [3]:
def encode(batches) -> None:
    encoder = ArrowToPostgresBinaryEncoder(arrow_table.schema)
    encoder.write_header()
    for batch in batches:
        encoder.write_batch(batch)
    encoder.finish()


In [4]:
%timeit encode(small_batches)

71.8 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
from time import time

start = time()
encode(large_batches)
end = time()
print(f"{end-start}")

28.74977469444275
